In [1]:
import copy
import fp.rollout
from fireplace.exceptions import GameOver
import logging
import random

In [2]:
logging.disable(logging.INFO)
#logging.basicConfig(level=logging.WARNING)
#root_logger = logging.getLogger()
#root_logger.disabled = True

game = fp.rollout.prepare_game()

def run_copy():
    try:
        game_copy = copy.deepcopy(game)
        fp.rollout.play_full_game(game_copy)
    except GameOver:
        print("Game completed normally.")

%timeit -n5 run_copy()

Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
Game completed normally.
5 loops, best of 3: 148 ms per loop


In [ ]:
logging.disable(logging.INFO) #logging.NOTSET
random.seed(1857)
%timeit -n50 fp.rollout.test_full_game()

### %%prun -s stdname -T rollout_stats_new.txt

logging.disable(logging.INFO)
random.seed(1857)

for _ in range(50):
    fp.rollout.test_full_game()


In [ ]:
import timeit
timeit.repeat('import fp; import logging; logging.disable(logging.INFO); fp.rollout.test_full_game()', number=50, repeat=3)